In [247]:
# pip install psycopg2

import pandas as pd
import psycopg2

Conexão com o servidor

In [248]:
server = 'localhost'
database = 'python_etl'
user = 'postgres'
password = 'root'

connection_string = f"dbname={database} user={user} password={password} host={server}"

try:
    conn = psycopg2.connect(connection_string)
    print("Conexão bem-sucedida ao PostgreSQL!")
except Exception as e:
    print("Erro ao conectar ao PostgreSQL:", e)

Conexão bem-sucedida ao PostgreSQL!


In [249]:
dados = pd.read_excel("/home/juniorsilvadev/engenharia-dados/python-etl/Origem/Origem/arquivos_excel/Produto.xlsx")
dados.head(5)

,ID,Name,Price,Id_Category
0,0,Sapato vermelho,96,0
1,1,Bolsa mais que Velha,8000,0
2,2,Calça RASGADA,75,0
3,3,Camisa apertadinha,170,0
4,4,Vestido Super curto,57,0


In [250]:
# Verificar os tipos de dados
print(dados.dtypes)

ID              int64
Name           object
Price           int64
Id_Category     int64
dtype: object


In [251]:
# Verificar os nomes das colunas
dados.columns

Index(['ID', 'Name', 'Price', 'Id_Category'], dtype='object')

In [252]:
str(dados.columns).replace("'", "")

'Index([ID, Name, Price, Id_Category], dtype=object)'

In [253]:
# Converter tipos de dados (se necessário)
dados['ID'] = dados['ID'].astype(int)
dados['Price'] = dados['Price'].astype(float)
dados['Id_Category'] = dados['Id_Category'].astype(int)

In [254]:
# Renomear as colunas para nomes mais consistentes
dados = dados.rename(columns={
    'ID': 'id',
    'Price': 'price',
    'Id_Category': 'id_category',
    'Name': 'name'
})

print(dados.head())

   id                  name   price  id_category
0   0       Sapato vermelho    96.0            0
1   1  Bolsa mais que Velha  8000.0            0
2   2         Calça RASGADA    75.0            0
3   3    Camisa apertadinha   170.0            0
4   4   Vestido Super curto    57.0            0


In [255]:
# Remover registros nulos
dados = dados.dropna()

# Remover duplicatas pelo ID
dados = dados.drop_duplicates(subset=['id'])

In [256]:
# Criar tabela se caso não existir
create_table_query = """
CREATE TABLE IF NOT EXISTS produtos (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    price NUMERIC(10, 2),
    id_category INTEGER
);
"""

# Reinicie a conexão, se necessário
conn.close()
conn = psycopg2.connect(connection_string)

try:
    cursor = conn.cursor() # Continuar com o cursor
    cursor.execute(create_table_query)
    conn.commit() # Commit após inserir todos os dados
    
    print("Tabela 'produtos' criada ou já existente.")
except Exception as e:
    print("Erro ao criar a tabela:", e)
    conn.rollback()

Tabela 'produtos' criada ou já existente.


In [257]:
# Executar sessão de apagar dados
cursor.execute('TRUNCATE TABLE produtos')

In [258]:
# Executar sessão de inserção dados na tabela
insert_query = """
INSERT INTO produtos (
    id, 
    name, 
    price, 
    id_category) VALUES (%s, %s, %s, %s)
"""

In [259]:
try:
    # Verifique se a conexão está aberta
    if conn.closed:
        conn = psycopg2.connect(connection_string)
        
    cursor = conn.cursor() # Continuar com o cursor
    
    # Preparar os dados como uma lista de tuplas
    data_to_insert = [
        (row['id'], row['name'], row['price'], row['id_category'])
        for index, row in dados.iterrows()
    ]
    
    # Inserir os dados em massa
    cursor.executemany(insert_query, data_to_insert) 

    conn.commit() # Commit após inserir todos os dados
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Erro ao inserir dados:", e)
    conn.rollback()  # Reverter alterações em caso de erro
finally:
    cursor.close() # Fecha o cursor
    conn.close() # Fecha a conexão

Dados inseridos com sucesso!
